<a href="https://colab.research.google.com/github/vishnubharadwaj1234/AI-Generated-Faces/blob/main/AIGeneratedFaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports + Mounting Google Drive

In [ ]:
import pandas as pd
import numpy as np
import os, random, cv2
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Dropout, Activation, MaxPool2D
from keras.optimizers import Adam, RMSprop
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from keras.applications import VGG19
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/AIGeneratedFacesProject/FacesData/'

# Getting the Data + Data Preprocessing

In [ ]:
!cp /content/drive/MyDrive/AIGeneratedFacesProject/FacesData/RealFaces.zip /content/

In [ ]:
!unzip /content/RealFaces.zip

In [ ]:
imageArrays = []
i = 0
for dirname, _, filenames in os.walk('/content/real_faces_128'):
    for filename in filenames:
        filePath = os.path.join(dirname, filename)
        imageArrays.append(cv2.imread(filePath))
        i += 1
        if i == 2000:
            break

In [ ]:
imageBatch = np.array(imageArrays)
print(imageBatch.shape)

In [ ]:
!cp /content/drive/MyDrive/AIGeneratedFacesProject/FacesData/FakeFaces.zip /content/

In [ ]:
!unzip /content/FakeFaces.zip

In [ ]:
fakeImageArrays = []
i = 0
for dirname, _, filenames in os.walk('/content/fake_faces_70k'):
    for filename in filenames:
        filePath = os.path.join(dirname, filename)
        fakeImageArrays.append(cv2.imread(filePath))
        i += 1
        if i == 2000:
            break

In [ ]:
fakeImageBatch = np.array(fakeImageArrays)
print(fakeImageBatch.shape)

In [ ]:
yRealImages = np.ones((imageBatch.shape[0]))
yFakeImages = np.zeros((fakeImageBatch.shape[0]))
y = np.concatenate((yRealImages, yFakeImages), axis=0)
print(y.shape)

In [ ]:
X = np.concatenate((imageBatch, fakeImageBatch), axis=0)
print(X.shape)

In [ ]:
X_norm_imgs = []
def normalize_one_image(image):
  return (image - np.min(image)) / (np.max(image) - np.min(image))
for idx in range(X.shape[0]):
  X_norm_imgs.append(normalize_one_image(X[idx]))
X_norm_imgs = np.array(X_norm_imgs)

In [ ]:
X_norm_imgs.shape

In [ ]:
X_norm_images_flat = np.reshape(X_norm_imgs, (X_norm_imgs.shape[0], -1))

In [ ]:
X_norm_images_flat.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm_images_flat, y, test_size=0.2, random_state=0, shuffle=True)

# Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)

In [ ]:
print("Accuracy: " + str(accuracy_score(y_test, predictions)))
print("Precision: " + str(precision_score(y_test, predictions)))
print("Recall: " + str(recall_score(y_test, predictions)))
print("F1 Score: " + str(f1_score(y_test, predictions)))

# MLP Classifier

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), random_state=1, max_iter=100)
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)

In [ ]:
print("Accuracy: " + str(accuracy_score(y_test, predictions)))
print("Precision: " + str(precision_score(y_test, predictions)))
print("Recall: " + str(recall_score(y_test, predictions)))
print("F1 Score: " + str(f1_score(y_test, predictions)))

# VGG19

In [ ]:
model_vgg19 = VGG19 (
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(128, 128, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

model = Sequential();
for layer in model_vgg19.layers:
  model.add(layer)

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))

num_layers = len(model.layers)
for x in range(0,num_layers):
    print(model.layers[x])
    print(model.layers[x].trainable)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, metrics=['accuracy'], loss='binary_crossentropy')

In [ ]:
checkpoint_path = "/content/drive/MyDrive/AIGeneratedFacesProject/FacesSavedCheckpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                 verbose=1)

In [ ]:
X_train_VGG, X_test_VGG, y_train_VGG, y_test_VGG = train_test_split(X_norm_imgs, y, test_size=0.9, random_state=0, shuffle=True)

In [ ]:
# run this if you want to train the model yourself
model.fit(X_train_VGG, y_train_VGG, epochs = 50000, callbacks=[cp_callback], )

In [ ]:
# run this to load the pretrained model
model.load_weights(checkpoint_path)

In [ ]:
predictions = model.predict(X_test_VGG)
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = 0
predictions = np.squeeze(predictions)

In [ ]:
y_test_VGG_int = y_test_VGG.astype(int)
classification_report(y_test_VGG_int, predictions)

# Vision Transformer